In [1]:
!git clone https://github.com/jaywonchung/BERT4Rec-VAE-Pytorch  # clone repo

Cloning into 'BERT4Rec-VAE-Pytorch'...
remote: Enumerating objects: 166, done.
remote: Total 166 (delta 0), reused 0 (delta 0), pack-reused 166
Receiving objects: 100% (166/166), 3.05 MiB | 1.91 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [1]:
%cd BERT4Rec-VAE-Pytorch/


/content/BERT4Rec-VAE-Pytorch


In [6]:
!pip install -r requirements.txt

In [2]:
!python main.py --template train_bert

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Input 1 for ml-1m, 20 for ml-20m: 1
0
Folder created: /content/BERT4Rec-VAE-Pytorch/experiments/test_2022-01-07_37
{'anneal_cap': 0.2,
 'bert_dropout': 0.1,
 'bert_hidden_units': 256,
 'bert_mask_prob': 0.15,
 'bert_max_len': 100,
 'bert_num_blocks': 2,
 'bert_num_heads': 4,
 'best_metric': 'NDCG@10',
 'dae_dropout': 0.5,
 'dae_hidden_dim': 600,
 'dae_latent_dim': 200,
 'dae_num_hidden': 0,
 'dataloader_code': 'bert',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'ml-1m',
 'dataset_split_seed': 98765,
 'decay_step': 25,
 'device': 'cuda',
 'device_idx': '0',
 'enable_lr_schedule': True,
 'eval_set_size': 500,
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'find_best_beta': False,
 'gamma': 1.0,
 'log_period_as_iter': 12800,
 'lr': 0.001,
 'met

In [4]:
!unzip /content/content.zip -d /content/BERT4Rec-VAE-Pytorch/experiments

Archive:  /content/content.zip
   creating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/
  inflating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/config.json  
   creating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/models/
  inflating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/models/checkpoint-recent.pth  
  inflating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/models/best_acc_model.pth  
   creating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/logs/
  inflating: /content/BERT4Rec-VAE-Pytorch/experiments/content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2/logs/events.out.tfevents.1627149185.98b0e63be087.360.0  


In [ ]:
!zip -r /content/folder /content/BERT4Rec-VAE-Pytorch/experiments/test_2021-07-24_2

In [8]:
!pip uninstall pandas
!pip install pandas==0.25.0

Found existing installation: pandas 0.25.0
Uninstalling pandas-0.25.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-0.25.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
Proceed (y/n)? ERROR: Operation cancelled by user


In [1]:
%cd BERT4Rec-VAE-Pytorch/

/content/BERT4Rec-VAE-Pytorch


In [3]:
import sys
sys.argv = ['--template train_bert']
from main import *
from templates import *
# import subprocess
# proc = subprocess.Popen(['python', 'main.py',  '--template train_bert', '--mode train'])
print(sys.argv)
args.template = "test_bert"
set_template(args)
model,test_loader=train()

import torch

#from conversion import *
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import pickle
import pandas as pd
from types import SimpleNamespace


# Load best model file
export_root = 'experiments/test_2022-01-07_37'
best_model = torch.load(os.path.join(export_root, 'models', 'best_acc_model.pth')).get('model_state_dict')
print (list(best_model.keys()))
# for key in list(best_model.keys()):
#     best_model[key.replace('bert.', '')] = best_model.pop(key)


model.load_state_dict(best_model)
model.to("cuda")
model.eval()

# Get sample tensor to load data into the model
dataset = test_loader.dataset
trainset = DataLoader(dataset=dataset,
                      batch_size=1,
                      shuffle=False,
                      pin_memory=True)
data = tqdm(trainset)
ids_and_tensors = list(enumerate(data))

# Load dataset (to get item mapping)
export_root = '/content/BERT4Rec-VAE-Pytorch/Data/preprocessed/ml-1m_min_rating0-min_uc5-min_sc0-splitleave_one_out/dataset.pkl'
data = pickle.load(open(export_root, "rb"))
train = data['train']
val = data['val']
test = data['test']
umap = data['umap']
smap = data['smap']
user_count = len(umap)
item_count = len(smap)
inv_map = {v: k for k, v in smap.items()}
inv_user_map = {v: k for k, v in umap.items()}
user_count, item_count

# Function to get top predictions for a list of IMDB ids
""" get top predictions from the model based on IMDB ids """
def predict(input_movies, k=100):
    user_id = 1111 # arbitrary, doesn't matter
    source_movies = input_movies

    with torch.no_grad():
        movies = [smap[id] for id in input_movies]
        start = len(ids_and_tensors[user_id][1][0][0]) - len(movies)
        for x in range(0, len(ids_and_tensors[user_id][1][0][0])):
            ids_and_tensors[user_id][1][0][0][x] = 0
        for x in range(start, len(ids_and_tensors[user_id][1][0][0])):
            ids_and_tensors[user_id][1][0][0][x] = movies[x-start]
        scores = model(ids_and_tensors[user_id][1][0].to('cuda'))
        scores = scores[:, -1, :]
        predictions = torch.topk(scores, k=k)
        tops = predictions[1].cpu().numpy()[0]

    output_movies = []
    for x in range(0, len(tops)):
        output_movies.append(inv_map[tops[x]])

    final_results = [movie for movie in output_movies if movie not in source_movies]
    source, prediction = source_movies, final_results

    return source, prediction

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


['--template train_bert']
Input 1 for ml-1m, 20 for ml-20m: 1
0
Folder created: /content/BERT4Rec-VAE-Pytorch/experiments/test_2022-01-07_38
{'anneal_cap': 0.2,
 'bert_dropout': 0.1,
 'bert_hidden_units': 256,
 'bert_mask_prob': 0.15,
 'bert_max_len': 100,
 'bert_num_blocks': 2,
 'bert_num_heads': 4,
 'best_metric': 'NDCG@10',
 'dae_dropout': 0.5,
 'dae_hidden_dim': 600,
 'dae_latent_dim': 200,
 'dae_num_hidden': 0,
 'dataloader_code': 'bert',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'ml-1m',
 'dataset_split_seed': 98765,
 'decay_step': 25,
 'device': 'cuda',
 'device_idx': '0',
 'enable_lr_schedule': True,
 'eval_set_size': 500,
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'find_best_beta': False,
 'gamma': 1.0,
 'log_period_as_iter': 12800,
 'lr': 0.001,
 'metric_ks': [1,
               5,
               10,
               20,
               50,
               100],
 'min_rating': 0,
 'min_sc': 0,
 'min_uc': 5,
 'mode': 'train',
 'model_code': 'bert',

  0%|          | 579/204569 [00:00<00:35, 5783.41it/s]

['bert.embedding.token.weight', 'bert.embedding.position.pe.weight', 'bert.transformer_blocks.0.attention.linear_layers.0.weight', 'bert.transformer_blocks.0.attention.linear_layers.0.bias', 'bert.transformer_blocks.0.attention.linear_layers.1.weight', 'bert.transformer_blocks.0.attention.linear_layers.1.bias', 'bert.transformer_blocks.0.attention.linear_layers.2.weight', 'bert.transformer_blocks.0.attention.linear_layers.2.bias', 'bert.transformer_blocks.0.attention.output_linear.weight', 'bert.transformer_blocks.0.attention.output_linear.bias', 'bert.transformer_blocks.0.feed_forward.w_1.weight', 'bert.transformer_blocks.0.feed_forward.w_1.bias', 'bert.transformer_blocks.0.feed_forward.w_2.weight', 'bert.transformer_blocks.0.feed_forward.w_2.bias', 'bert.transformer_blocks.0.input_sublayer.norm.a_2', 'bert.transformer_blocks.0.input_sublayer.norm.b_2', 'bert.transformer_blocks.0.output_sublayer.norm.a_2', 'bert.transformer_blocks.0.output_sublayer.norm.b_2', 'bert.transformer_blocks.

100%|██████████| 204569/204569 [00:43<00:00, 4727.30it/s]


In [6]:
x,y = predict([30831,32937,29803,31240,21])
df=pd.DataFrame(y)
df[:10]

,0
0,25777
1,32282
2,22535
3,30240
4,31043
5,32281
6,34134
7,30276
8,22199
9,19815
